Imports

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, f1_score

Data mounting

In [ ]:
# Data mounting
from google.colab import drive
drive.mount('/content/drive')

data_dir = '/content/drive/MyDrive/Fabric_Defect_Dataset' 

Transformer 

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder(root=data_dir, transform=transform)

Splitting of data

In [ ]:
# Splitting the data
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

ResNet feature extraction and classifier(for embedding)

In [ ]:
#ResNet
class FabricDefectModel(nn.Module):
    def __init__(self, num_classes):
        super(FabricDefectModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        
        #Freezing the ResNet layer
        for param in self.resnet.parameters():
            param.requires_grad = False

        #Storing no. of input features before replacing the fc layer
        in_features = self.resnet.fc.in_features

        #Final layer -> Identity layer
        self.resnet.fc = nn.Identity()

        #Classifier
        self.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
            nn.Softmax(dim=1)
        )
                                        

    def forward(self, x):

        #Embedding extraction
        features = self.resnet(x)

        return self.classifier(features)

In [ ]:
num_classes = len(dataset.classes)
model = FabricDefectModel(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    best_val_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

       

        for images, labels in train_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)

        model.eval()
        val_preds = []
        val_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pth')
            print('Model saved with improved validation accuracy!')

Evaluation

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    test_preds = []
    test_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')



    test_accuracy = accuracy_score(test_labels, test_preds)
    test_f1_score = f1_score(test_labels, test_preds, average='weighted')

    return test_accuracy, test_f1_score

Train

In [ ]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25)

Accuracy

In [ ]:

#Accuracy
model.load_state_dict(torch.load('best_model.pth'))
test_accuracy, test_f1_score = evaluate_model(model, test_loader)

print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test F1 Score: {test_f1_score:.4f}')